In [1]:
import os
import sys
import numpy as np
import pandas as pd

sys.path.append('../utils')
from strategy_funcs import process_dataframe
from helper_functions import triple_barrier, curado_profits

In [2]:
def ensure_utc(df, col_names):
    """
    Ensure the datetime columns are in UTC.
    """
    if isinstance(col_names, str):  # If a single column is passed as a string convert to list for uniform processing
        col_names = [col_names]     
    
    for col_name in col_names:
        df[col_name] = pd.to_datetime(df[col_name])
        if df[col_name].dt.tz is None:
            df[col_name] = df[col_name].dt.tz_localize('UTC')  # Localize to UTC if no timezone
        else:
            df[col_name] = df[col_name].dt.tz_convert('UTC')  
            

def prepare_data(df, SL=0.025, TP=0.05, tl=48, window_size=28):
    """
    Run labeling method and process dataframe 
    """
    df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)

    # Labeling
    data_labeled = triple_barrier(df, SL=SL, TP=TP, tl=tl).reset_index(drop=True)
    data_labeled['profit_curado'] = curado_profits(data_labeled)
    data_labeled = data_labeled.drop(columns=['open', 'high', 'low', 'close', 'volume', 'label'])

    # Process by windows
    data_processed = process_dataframe(df, lags=window_size, cols=['open', 'high', 'low', 'close', 'volume'])
    data_processed = data_processed.reset_index().rename(columns={'date': 'startDate'})
    
    # Add labels to processed dataframe
    profits_df = pd.merge(data_processed, data_labeled, on='startDate', how='inner')
    ensure_utc(profits_df, ['startDate', 'endDate'])

    return profits_df

In [3]:
path = '~/freqtrade/user_data/data/binance/'
coins = ['BTC', 'BNB', 'ETH', 'DOGE', 'ADA', 'SOL', 'MATIC']
lista_df = []
tf = '1h'
SL = None
TP = 0.03
tl = 48
window_size = 48

for coin in coins:
    print(coin)
    dir_coin = path + f'{coin}_USDT-{tf}.feather'
    df_coin = pd.read_feather(dir_coin)
    df_prep = prepare_data(df_coin, SL=SL, TP=TP, tl=tl, window_size=window_size).sort_values(by='startDate')
    df_prep['pair'] = coin
    lista_df.append(df_prep)

df_final = pd.concat(lista_df, axis=0)

BTC


100%|███████████████████████████████████████████| 61409/61409 [00:07<00:00, 8024.78it/s]


BNB


100%|███████████████████████████████████████████| 61409/61409 [00:07<00:00, 8718.46it/s]


ETH


...

100%|███████████████████████████████████████████| 58895/58895 [00:06<00:00, 8692.73it/s]


SOL


100%|███████████████████████████████████████████| 38633/38633 [00:04<00:00, 8538.74it/s]


MATIC


100%|███████████████████████████████████████████| 45765/45765 [00:05<00:00, 8581.22it/s]


In [9]:
df_final.to_csv('~/Downloads/crypto48h_3TP.csv', index=False)